### Diversity analysis

This notebook will analyse the diversity of AI research.

This entails:

* Loading the data
* Estimating diversity based on agglomerative clustering
* Comparing diversities at different points
* Comparing diversities of research involving different groups (eg universities vs companies, research involving women and not involving women)

## 0. Preamble

In [ ]:
%run notebook_preamble.ipy

In [ ]:
# Ignore future warnings (for when I concatenate dfs)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Other imports

In [ ]:
import random

from statsmodels.api import OLS, Logit
from statsmodels.tools.tools import add_constant
from scipy.stats import zscore
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from scipy.stats import entropy
import seaborn as sns


### Functions

Add a bunch of exogenous variables to the analysis df

In [ ]:
#Generic functions
def save_fig(name,path='../reports/figures/paper_rev/'):
    '''
    Saves a figure
    '''
    plt.tight_layout()
    
    plt.savefig(path+f'{today_str}_{name}')
    
    # Put functions etc here

def flatten_list(my_list):
    '''
    Flattens a list
    '''
    
    return([x for el in my_list for x in el])


def get_example(df,number,length):
    '''
    Gets random examples in a field
    
    Args:
        Df is the dataframe we want to use
        number is the number of examples we want
        length is the length of the examples
    
    '''
    
    choose = random.sample(list(df.index),number)
    
    for x in df.loc[choose]['abstract']:
        
        print(x[:length])
        print('\n')

In [ ]:
def make_tidy_lookup(names_list,length=False):
    '''
    
    Creates a cheap lookup between names, removing underscores and capitalising
    
    Args:
        names_list (list) is the list of names we want to tidy
        length is if we want to only keep a certain length of the name
    
    '''
    
    out = {x:re.sub('_',' ',x).capitalize() for x in names_list}
    return(out)



In [ ]:
def cross_sectional_comp(df,variable,topics,threshold):
    '''
    This function compares activity by topics between categories.
    
    Args:
        df is the dataframe we are using (generally analysis_fin, with rows = papers and columns = variables and metadata)
        variable is the variable we are using for the comparison
        topics is the topics where we want to compare (generally the community names)
        threshold is the threshold we want to use to determine if a paper is in a topic or not
    
    Returns a df with the shares of papers in each topic sorted by their distances
    
    '''
    
    #Create the counts df.
    
    #We are extracting, for each topics, the % of papers with at least one female author when the topic is present, and when it isn't.
    group_counts = pd.concat([pd.crosstab(df[variable],df[t]>threshold,normalize=1).loc[True,:] for t in topics],axis=1)
    
    #Name
    group_counts.columns = topics
    
    #Transpose
    group_counts = group_counts.T
    
    #Rename variables
    group_counts.columns = [variable+f'_{value}' for value in ['false','true']]
    
    #Create a measure of difference
    group_counts['difference'] = (group_counts.iloc[:,1]/group_counts.iloc[:,0])-1
    
    #Output
    out = group_counts.sort_values('difference',ascending=False)
    
    return(out)

def topic_regression(df,target_list,exog,controls,model,binarise=False,standardise=True,cov='HC1'):
    '''
    
    This function regresses topic weights (or their binarisation) on predictors.
    
    Arguments:
        -Df with the variables
        -target_list: target variables. This is a list we loop over. 
        -exog: exogenous variable
        -controls
        -model type. OLS? Logit? TODO fix the logit
        -Binarise in case we are using logit. If not False, the value is the threshold 
            TODO when we binarise the highly detailed models, some of them become all zeros. This will work better
            with the mopre aggregate topics
        -Standardise if we standardise and log the topic weights
    
    Returns
        -A list of statsmodels summaries

    
    '''
    
    #Drop rows with missing values - sm doesn't like them
    df_2 = df[target_list+exog+controls].dropna(axis=0)
    
    #Standardise targets?
    if standardise==True:
        df_2[target_list] = (np.log(df_2[target_list]+0.00000001)).apply(zscore).astype(float)
    
    #Binarise targets if we are doing a logit
    if binarise!=False:
        df_2[target_list] = df_2[target_list].applymap(lambda x: x>binarise).astype(float)
    
    
    #Extract the exogenous and controls, add constant and cast as float
    exog_controls = add_constant(df_2[exog+controls]).astype(float)
    

    #Container output
    out = []
    coeffs = []
    
    #One regression for each target
    for t in list(target_list):
        
        #There we gp. 
        reg = model(endog=df_2[t],exog=exog_controls).fit(cov_type=cov,disp=0)
        
        out.append(reg.summary())
        
        #coeffs.append(reg)
        if model == OLS:
            coeffs.append(pd.Series([float(reg.params[exog]),float(reg.pvalues[exog]),float(reg.rsquared)],name=t))
            reg_coeff = pd.concat(coeffs,axis=1).T
            reg_coeff.columns = ['coefficient','p_value','r_square']
    
        else:
            coeffs.append(pd.Series([float(reg.params[exog]),float(reg.pvalues[exog]),float(reg.prsquared)],name=t))
            reg_coeff = pd.concat(coeffs,axis=1).T
            reg_coeff.columns = ['coefficient','p_value','pr_square']
 
    
    return([out,reg_coeff.sort_values('coefficient',ascending=False)])
        
       

def plot_regression_coefficients(df,var,cov='HC1',size=(8,6),ax=False,ncols=3):
    '''
    Plots regression coefficients.
    
    Arg:
        variable we use as predictor.
    
    '''
    
    reg = topic_regression(df,topics,[var],controls,OLS,cov='HC1')
    
    if ax==False:
        fig,ax = plt.subplots(figsize=size)

    plot_topic_bar(reg[1]['coefficient'],cl=color_lookup,ax=ax,ncols=ncols)

    ax.set_title(f'Regression coefficient using {var} as predictor')

def topic_comparison(df,target_list,exog,concept_lookup,quantiles=np.arange(0,1.1,0.2),thres=0):
    '''
    This function compares the distribution of activity in various topics depending on an exogenous variable of interest. 
    
    Args:
        Df with the topic mix and metadata
        target_list are the topics to consider
        exog is the variable to crosstab topics against
        concept_lookup is a df with the median proximity of each topic to the concepts
        quantiles is how we discretise the concept lookup (default value is quintiles)
        thres: =limit for considering a topic as present

    
    '''
    
    #Copy df
    
    df_2 = df.copy()
    
    #Discretise the concept lookup
    
    conc_discr = concept_lookup.apply(lambda x: pd.qcut(x,q=quantiles,labels=False,duplicates='drop'))

    
    #Calculate levels of activity per topic based on the exog variable
    
    topic_distr = pd.concat([pd.crosstab(df_2[exog],df_2[t]>thres)[True] for t in target_list],axis=1).T
    topic_distr.index = target_list
    
    
    #Merge the count with the concept lookup
    disc = pd.melt(pd.concat([topic_distr,conc_discr],axis=1).reset_index(drop=False),id_vars=['index']+list(conc_discr.columns))
    
    #This is the list where we store the results
    store={}
    
    for c in concept_lookup.columns:
        
        out = pd.pivot_table(disc.groupby([c,'variable'])['value'].sum().reset_index(drop=False),index=c,columns='variable',values='value')
        #out.apply(lambda x: x/x.sum()).plot.bar()
        
        store[c] = out
                                      
    #Output dfs with the comparisons
    return(store)

def plot_topic_bar(table,cl,ax,ncols):
    '''
    Simple function to plot topic bars which includes colours based on the topic-label lookup
    
    Args:
        table has topics in the index and a value to plot in the columns
        cl is the colour lookup between communities and topics
        ax is the plotting axe
    
    
    '''
    
    cols = [cl[comm_names[comms[x]]] if comm_names[comms[x]] in cl.keys() else 'lightgrey' for x in table.index]
    
    table.plot.bar(color=cols,ax=ax,width=1)
    
    ax.legend(handles=patches,ncol=ncols)
    ax.set_xticks([])
    ax.set_xticklabels([])
    
    
def calculate_entropy(df,categories,category):
    '''
    We calculate entropy inside a paper using a distribution over semantic variables (eg discipline, community or topic). These have to be normalised
    
    arguments:
        df is the analysis df with relevant topics and metadata
        categories are the topics we want to compare
        
    outputs
        A df with entropy measures by paper
        
    
    '''
    #Normalise
    norm = df[categories].apply(lambda x: x/x.sum(),axis=1)
    
    ent = pd.DataFrame((norm.apply(lambda x: entropy(x),axis=1)),columns=['entropy'])
    
    ent['cat']=category
    
    return(ent)

def make_exog(df,value_container,value,make_dummy=True):
    '''
    This creates exogenous variables for modelling later.
    
    Argument:
        -df contains the variable where we want to find a value
        -variable_container is the column where we want to look for the value
        -value is the value we are looking for
        -make_dummy: if true it just counts if the value is present. If false, it counts how many times it happens. 
        
    Output
        -A df with the new column (named)
    
    
    '''
    
    df_2 = df.copy()
    
    #Create a tidy variable name
    column_name = re.sub(' ','_',value.lower())
    
    #If we want to create a dummy...
    if make_dummy == True:
        
        #We just look for it in the value container
        #There are some missing values so we have some control flow to manage that. 
        df_2[column_name] = [value in x if type(x)==list else np.nan for x in df_2[value_container]]
    
    else:
        
        #Otherwise, we count how many times it occurs
        #We deal with missing values ('non lists') as before
        df_2[column_name] = [x.count(value) if type(x)==list else np.nan for x in df_2[value_container]]
        
    return(df_2)
    

def extract_topic_trend(df,cat,year_lims=[2000,2019]):
    '''
    Extracts evolution of a share of a category in a topic of interest
    
    Args:
        df: the usual dataframe
        cat: the category we are interested in
        year_lims: first and last year to consider

    '''
    #rel_df = df.loc[df[cat]==True]
    
    out = pd.crosstab(df['year'],df[cat],normalize=0)
    
    return(out.loc[np.arange(year_lims[0],year_lims[1])])

def plot_topic_trend(df,cat,topics,ax,cmap,year_lims=[2000,2019],threshold=0.05,focus_topics=False,alpha=0.2):
    '''
    Plots topic trends (shares of a category in a topic)
    
    Args:
        df the usual dataframe
        topics: topics we want to display
        cat: the category of interest
        year_lims: first and last year to consider
    
    '''
    activity = []
    names = []
    
    #Use a loop to deal with cases where a category has no activity in a topic
    for t in topics:
        try:
            levels = extract_topic_trend(df.loc[df[t]>threshold],cat,year_lims)
            activity.append(levels[True])
            names.append(t)
        
        except:
            pass
        
        
    topic_trends = pd.concat(activity,axis=1).fillna(0)
    topic_trends.columns = names
    
    if focus_topics !=False:
        
        topic_lookup = {name:val for val,name in enumerate(focus_topics)}

        #Color map
        cols = plt.cm.get_cmap(cmap)

        #Create a vector of colors
        cols_to_show = [(0.5,0.5,0.5,alpha) if v not in topic_lookup.keys() else cols(topic_lookup[v]) for v in topic_trends.columns]

        #Plot
        (100*topic_trends.rolling(window=4).mean().dropna()).plot(color=cols_to_show,ax=ax,linewidth=3)

        #Fix the legend to focus on key topics
        hand,labs = ax.get_legend_handles_labels()

        ax.legend(bbox_to_anchor=(1,1),handles = [x[0] for x in zip(hand,labs) if x[1] in focus_topics],
                  labels=[x[1][:50] for x in zip(hand,labs) if x[1] in focus_topics])
    
    else:

        topic_trends.rolling(window=4).mean().dropna().plot(ax=ax)
        ax.legend(bbox_to_anchor=(1,1))
    

    

    

In [ ]:
def get_university_industry_collab_trends(df,variable,topic,threshold=0.05):
    '''
    Study university industry collaborations
    
    Args:
        df as usual
        variable is the collaboration variable we want to study
        topic the topic
        threshold is the threshold for accept a paper in a topic
    

    '''
    
    df_with_topic = df.loc[df[topic]>threshold]
    

    topic_collabs = (100*pd.crosstab(df_with_topic['year'],df_with_topic['university_industry_collab'],normalize=0))[True]
    
    
    return(topic_collabs)
    

## 1. Load data

`analysis_pack` contains the metadata and data that we serialised at the end of the `06` data integration notebook.

This includes:

* Community names for the communities (`index->community name`)
* Community indices for topics (`topic -> community index`)
* Filtered topic names (`topic names`)
* Network object with topic co-occurrences
* Analysis df
* arx is the enriched arXiv dataset



In [ ]:
with open('../data/processed/24_8_2019_analysis_pack.p','rb') as infile:
    analysis_pack = pickle.load(infile)

In [ ]:
comm_names = analysis_pack[0]
comms = analysis_pack[1]
topics = analysis_pack[2]
network = analysis_pack[3]
data = analysis_pack[4]
arx = analysis_pack[5]

In [ ]:
color_lookup = {
    'deep_learning':'blue',
    'robotics_agents':'cornflowerblue',
    'computer_vision':'aqua',
    'symbolic':'red',
    'health':'lime',
    'social':'forestgreen',
    'technology':'magenta',
    'statistics':'orange',
    'language':'yellow'
}

#These are the field names
field_names = ['field_astrophysics',
 'field_biological',
 'field_complex_systems',
 'field_informatics',
 'field_machine_learning_data',
 'field_materials_quantum',
 'field_mathematical_physics',
 'field_mathematics_1',
 'field_mathematics_2',
 'field_optimisation',
 'field_particle_physics',
 'field_physics_education',
 'field_societal',
 'field_statistics_probability']

#Create tidy field names for legend etc
tidy_field_lookup = {x:re.sub('_',' ',x[6:]).capitalize() for x in field_names}

community_names = [x for x in list(set((comm_names.values()))) if x!='mixed']

tidy_comms_lookup = make_tidy_lookup(community_names)

patches = [mpatches.Patch(facecolor=c, label=tidy_comms_lookup[l],edgecolor='black') for l,c in color_lookup.items()]

### Some minor processing

In [ ]:
#Variables of interest
interesting = [['type_list','Company'],['type_list','Government'],['type_list','Education']]
              # ['institute_list_2','Google'],['institute_list_2','Facebook'],['institute_list_2','IBM'],['institute_list_2','Microsoft'],
              #['institute_list_2','Apple'],['institute_list_2','Amazon']]

#Create the expanded df
data_2 = data.copy()

#For each interesting variable we expand the df
for detect in interesting:
    
    data_2 = make_exog(data_2,value_container=detect[0],value=detect[1])

In [ ]:
data_2['only_company'] = (data_2['company']==True) & (data_2['education']==False)

### Analysis

We want to analyse the diversity of AI research. 

Questions:

* What definition of diversity do we use?
* What definition of distance do we use?




In [ ]:
from sklearn.metrics import pairwise_distances
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import cophenet

In [ ]:
#This is to be able to colour nodes based on their communities and their position in the chart
#topic_lookup = {n:t for n,t in enumerate(topics)}

color_lookup_names = {}

for t in topics:
    
    comm_name = comm_names[comms[t]]
    
    
    c = color_lookup[comm_name] if comm_name in color_lookup.keys() else 'grey'
    
    color_lookup_names[t] = c

Create a dendrogram that colours links based on their topic

In [ ]:
def tree_diversity(data,topics,metric,plot=False):
    '''
    This function creates a hierarchical tree based on distances between topics and calculates diversity (the length in the branches of that tree)
    
    Args:
        data (df) is the topic mix for each paper
        topics (list) are the topics to consider
        metric (str) is the metric of distance that we use
        plot (bool) is the boolean for whethet to plot or not
    
    Returns the measure of distance and (perhaps) a plot
    
    '''
    
    #Plot distances
    dists = hierarchy.linkage(data[topics].T,metric=metric)
    
    #This gives the presence of a topic in the population of papers. Currently using sum but it could be something else
    topic_length = data[topics].sum()
    
    #If no plot
    
    if plot==False:
        dend = hierarchy.dendrogram(dists,no_plot=True)
        
        #This gives the length of each branch at merge
        lengths = [x[1] for x in dend['dcoord']]
        
        path_length = np.sum(lengths)
        
        return(path_length)                       
        
    else:
        #Create axes etc
        fig,ax = plt.subplots(figsize=(6,12),ncols=2)

        dend = hierarchy.dendrogram(dists,
                                   labels=topics,
                                   ax=ax[0],orientation='left')
        
        #Labels to reorder the other plot 
        labels = [t.get_text() for t in ax[0].get_yticklabels()]

        ax[0].get_yaxis().set_visible(False)

        colors = [color_lookup_names[x] for x in labels]

        #Plot topic activity
        topic_length.loc[labels].plot.barh(ax=ax[1],color=colors,edgecolor='grey',linewidth=0.1)

        #Remove axis
        ax[1].get_yaxis().set_visible(False)

        #This gives the length of each branch at merge
        lengths = [x[1] for x in dend['dcoord']]
        
        path_length = print(np.sum(lengths))
        
        return(path_length)

#### Comparisons

Here we compare the topic mix of papers with different compositions

We will focus on a specific country and year to rule out compositional effects


In [ ]:
y = 2015

m = 'cosine'

country = 'United States'

data_temp = data_2.dropna(axis=0,subset=['country_list'])

data_3 = data_temp.loc[(data_temp['year']>y)&([country in c for c in data_temp['country_list']])]


#### Compare papers with and without women

In [ ]:
# With and without women

In [ ]:
div_fem = [tree_diversity(d,topics,m) for d in [data_3,
                                                       data_3.loc[data_3['has_female']==False]]]

div_fem

#### Compare corporate and non-corporate papers

In [ ]:
div_corp = [tree_diversity(d,topics,m) for d in [data_3,data_3.loc[data_3['company']==True]]]

div_corp

#### Compare papers in different periods

In [ ]:
m = 'cosine'

country = 'United States'

data_temp = data_2.dropna(axis=0,subset=['country_list'])

data_4 = data_temp.loc[[country in c for c in data_temp['country_list']]]

y_thres=2017


In [ ]:
div_year = [tree_diversity(d,topics,m) for d in [data_4.loc[data_4['year']<y_thres],data_4.loc[data_4['year']>y_thres]]]

div_year

#### What happens if we remove various topics?

In [ ]:
m = 'cosine'

div_base = tree_diversity(data_3,topics,m)

In [ ]:
div_top = pd.Series([div_base-tree_diversity(data_3,
                          [x for x in topics if x not in [k for k,v in comms.items() if v==comm_ind]],
                          m) for comm_ind,comm_name in comm_names.items()],index=comm_names.values())

In [ ]:
div_top.sort_values(ascending=False).plot.bar(figsize=(10,5))